In [54]:
from mpl_toolkits.mplot3d import Axes3D
from numpy import *
import matplotlib; 
from matplotlib.pyplot import *
#import seaborn as sns
#sns.set()
%matplotlib notebook 

Euler Scheme with variable step
------------

In [42]:
def solve_ivp_euler_explicit_variable_step(f, t0, x0, t_f, dtmin = 1e-16, dtmax = 0.1, atol = 1e-3):
    dt = dtmax/10; # initial integration step
    ts, xs = [t0], [x0]  # storage variables
    t = t0
    ti = 0  # internal time keeping track of time since latest storage point : must remain below dtmax
    x = x0
    while ts[-1] < t_f:
        while ti < dtmax:
            t_next, ti_next, x_next = t + dt, ti + dt, x + dt * f(x)
            x_back = x_next - dt * f(x_next)
            ratio_abs_error = atol / (linalg.norm(x_back-x)/2)
            dt = 0.9 * dt * sqrt(ratio_abs_error)
            if dt < dtmin:
                raise ValueError("Time step below minimum")
            elif dt > dtmax/2:
                dt = dtmax/2
            t, ti, x = t_next, ti_next, x_next
        dt2DT = dtmax - ti # time left to dtmax
        t_next, ti_next, x_next = t + dt2DT, 0, x + dt2DT * f(x)
        ts.append(t_next); xs.append(x_next)
        t, ti, x = t_next, ti_next, x_next
    return (array(ts), array(xs).T)


Pour l'adaptation du pas : on sait que l’erreur commise par le schéma d’Euler
$$
x^{j+1} = x^j + \Delta t_j f(t_j,x^j)
$$

à chaque itération est donné par (développement de Taylor)
$$
e^{j+1} = \frac{\Delta t_j^2}{2} x''(t_j) + O(\Delta t_j^3)
$$

i.e. par le calcul (1) page 20,
$$
e^{j+1} = \frac{\Delta t_j^2}{2}\Big( \partial_t f(t_j,x(t_j)) + \partial_x f(t_j,x(t_j)) f(t_j,x(t_j)) \Big)+ O(\Delta t_j^3)
$$

On conclut 2 choses :

1) D’après la correction de la question 1 de l’exo « Consistance de schéma » en haut de la page 21,
$$
f(t_{j+1},x^{j+1}) = f\Big(t_{j+1},x^j+\Delta t f(t_j,x^j) \Big) =  f(t_j,x^j) + \Delta t \Big( \partial_t f(t_j,x^j) + \partial_x f(t_j,x^j) f(t_j,x^j) \Big) + \mathrm{O}(\Delta t^2).
$$
Donc on a bien
$$
e^{j+1} = \frac{\Delta t}{2}\Big(f(t_{j+1},x^{j+1})- f(t_j,x^j) \Big)+ O(\Delta t^3)
$$
On peut donc estimer en ligne $|e_{j+1}|$ par $x_{back} = x^{j+1} - \Delta t f(t_{j+1},x^{j+1})$. Dans le code, j’ai écrit ça un peut différemment en observant que
$$
\frac{\Delta t}{2}\Big(f(t_{j+1},x^{j+1})- f(t_j,x^j) \Big) = \frac{1}{2} (x^j - x_{back})
$$
où xback est l’estimée en temps rétrograde à partir de xj+1 , ie.
$$
x_{back} = x^{j+1} - \Delta t f(t_{j+1},x^{j+1})
$$
Donc en gros, on fait Euler en forward puis en backward et on regarde la différence avec le point initial xj .

2) Une fois qu’on a estimé l’erreur commise ej+1 , on sait que $e^{j+1} = O(\Delta t^2)$ donc si on veut avoir une erreur de l’ordre de Tolabs , on adapte le pas en $\Delta t_{j+1} = \Delta t_j \sqrt{\frac{\texttt{Tol}_{abs}}{\|e^{j+1}\|}}$